In [1]:
!pip install seaborn -q

In [2]:
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss, roc_auc_score

import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import datetime as dt
import warnings
import seaborn as sns

%matplotlib inline
warnings.filterwarnings('ignore')

Выполним предобработку данных и EDA аналогично ДЗ 1 (только дропать будем уже другие столбцы, согласно условию)

In [3]:
np.random.seed(10)

In [4]:
path_to_data = '../input/clicks/data.csv'

In [ ]:
data = pd.read_csv(path_to_data)
data.head()

In [ ]:
data.describe()

Как видим, impressions состоит только из одних единиц, т.к. этот признак в себе не несет информации, можем его отбросить

In [ ]:
def drop_unnecessary_data(data: pd.DataFrame) -> pd.DataFrame:
    return data.drop(['oaid_hash', 
                     'campaign_clicks', 
                      'impressions'], axis=1)

In [ ]:
def plot_features(features: pd.DataFrame):
    fig, axs = plt.subplots(features.shape[1] // 2, 2, figsize=(12,9))
    for i in tqdm(range(features.shape[1])):
        axs[i // 2, i % 2].hist(features.iloc[:, i])
        axs[i // 2, i % 2].set_title(str(features.columns[i]))
    plt.tight_layout()
    plt.show()

In [ ]:
def clean_data(data: pd.DataFrame) -> pd.DataFrame:
    cleaned_data = drop_unnecessary_data(data)
    cleaned_data.date_time = pd.to_datetime(cleaned_data.date_time)
    return cleaned_data

In [ ]:
def analysis(data: pd.DataFrame):
    cleaned_data = clean_data(data)
    print(f'Размер датасета: {cleaned_data.shape}')
    print(f'NaN значений в датасете: {cleaned_data.isna().values.sum()}')
    print(f'Уникальных значений в столбцах: \n{cleaned_data.nunique()}')
    plot_features(cleaned_data.iloc[:, 1:-1])
    print(f'Матрица корреляции:')

In [ ]:
analysis(data)

Можем сделать следующие выводы:

* В датасете на этот раз присутствуют `NaN` значения (однако в масштабе всего датасета их не столь много)

* Много категориальных признаков для которых потребуется `OneHotEncoding`


In [ ]:
cleaned_data = clean_data(data).dropna()
cleaned_data.shape


Применим  `OneHotEncoding` , после чего поделим `X` и `y` на `train` и `test` (последний день)

In [ ]:
def get_X_y(data: pd.DataFrame) -> (pd.DataFrame, pd.Series):
    X = data.drop(columns=['clicks'])
    y = data.clicks
    return X, y

In [ ]:
def train_test_split(data: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame, pd.Series, pd.Series):
   
    last_year = data.date_time.max().year
    last_month = data.date_time.max().month
    last_day = data.date_time.max().day
    latest_data = pd.Timestamp(last_year, last_month, last_day)
    test_data = data[data.date_time >= latest_data]
    train_data = data[data.date_time < latest_data]
    train_data = train_data.drop(columns=['date_time'])
    test_data = test_data.drop(columns=['date_time'])
    
    X_train, y_train = get_X_y(train_data)
    X_test, y_test = get_X_y(test_data)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def feature_engineering(data: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame, pd.Series, pd.Series):    
    X_train, X_test, y_train, y_test = train_test_split(data)
    
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
    X_train = ohe.fit_transform(X_train)
    X_test = ohe.transform(X_test)
    
    return X_train, X_test, y_train, y_test

Кроме деления на `train` и `test` по дню, также оставим `test` для $\pi_0$ и $\pi_1$, т.к. они различаются по `banner_id`

In [ ]:
X_train, X_test, y_train, y_test = feature_engineering(cleaned_data)
# pi 0 and pi 1

Будем использовать как и в 1-ой ДЗ логистическую регрессию с liblinear optimizer и $L2$ регуляризацией при $C = 0.01$

In [ ]:
def create_model(C=0.01):
    return LogisticRegression(solver='liblinear', 
                              random_state=10,
                              penalty='l2',
                              C=C)

In [ ]:
log_reg_model = create_model()
log_reg_model.fit(X_train, y_train)

In [ ]:
model_pred = best_log_reg.predict_proba(X_test)

В качестве baseline модели воспользуемся средним по test выборке

In [ ]:
model_pred

In [ ]:
def get_stats(baseline_pred: np.array, model_pred: np.array, y_test: np.array):
    auc_baseline = roc_auc_score(y_test, baseline_pred)
    log_loss_baseline = log_loss(y_test, baseline_pred)
    auc_model = roc_auc_score(y_test, model_pred[:, 1])
    log_loss_model = log_loss(y_test, model_pred)
    print(f'Logistic regression model: ')
    print(f'----------------')
    print(f'Log Loss : {log_loss_model}')
    print(f'ROC AUC : {auc_model}')
    print(f'Baseline model: ')
    print(f'----------------')
    print(f'Log Loss : {log_loss_baseline}')
    print(f'ROC AUC : {auc_baseline}')

In [ ]:
get_stats(baseline_pred, model_pred, y_test)

Как видим, используя логистическую регрессию нам удалось превзойти бейзлайн решение для предсказания вероятности клика